## RSS 방식은 최근 50개만 가능해서 실패

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# ChromeDriverService를 사용하는 방법
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

def get_blog_post_addresses(blog_url):

    post_addresses = []  # 포스트 주소를 저장할 리스트

    try:
        # 네이버 블로그 메인 페이지를 엽니다
        driver.get(blog_url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

        # iframe으로 구성된 페이지일 경우, iframe 내부로 스위치합니다
        iframes = driver.find_elements(By.TAG_NAME, 'iframe')
        for iframe in iframes:
            driver.switch_to.frame(iframe)
            try:
                # 포스트 목록을 찾습니다
                posts = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, 'blog.naver.com/inno06039')]")))
                for post in posts:
                    post_url = post.get_attribute('href')
                    post_addresses.append(post_url)
            except TimeoutException:
                print("포스트를 찾는 중 시간 초과가 발생했습니다.")
            driver.switch_to.default_content()  # 기본 컨텐츠로 돌아갑니다

    except Exception as e:
        print(f"오류가 발생했습니다: {e}")
    finally:
        driver.quit()  # 드라이버를 종료합니다

    return post_addresses

# 블로그 URL
blog_url = "https://blog.naver.com/inno06039"
# 포스팅 주소를 가져옵니다
post_addresses = get_blog_post_addresses(blog_url)

# 결과 출력
for address in post_addresses:
    print(address)


https://nid.naver.com/nidlogin.login?mode=form&url=https://blog.naver.com/inno06039
https://nid.naver.com/nidlogin.login?mode=form&url=https://blog.naver.com/inno06039
https://nid.naver.com/nidlogin.login?url=https://blog.naver.com/inno06039
https://nid.naver.com/nidlogin.logout?returl=https://blog.naver.com/inno06039
https://rss.blog.naver.com/inno06039.xml
https://rss.blog.naver.com/inno06039.xml?rss=1.0
https://rss.blog.naver.com/inno06039.xml?atom=0.3


In [27]:
import feedparser

def get_blog_post_addresses_from_rss(rss_url):
    # RSS 피드 파싱
    feed = feedparser.parse(rss_url)
    
    # 포스트 주소를 저장할 리스트
    post_addresses = []
    
    # feed.entries에 있는 각 포스트의 링크 추출
    for entry in feed.entries:
        post_addresses.append(entry.link)
    
    return post_addresses

# RSS 피드 URL
rss_url = "https://rss.blog.naver.com/inno06039.xml"

# 포스팅 주소를 가져옵니다
post_addresses = get_blog_post_addresses_from_rss(rss_url)

# 결과 출력
for address in post_addresses:
    print(address)


https://blog.naver.com/inno06039/223315892193?fromRss=true
https://blog.naver.com/inno06039/223314262954?fromRss=true
https://blog.naver.com/inno06039/223313566231?fromRss=true
https://blog.naver.com/inno06039/223156524687?fromRss=true
https://blog.naver.com/inno06039/223143200809?fromRss=true
https://blog.naver.com/inno06039/223142177846?fromRss=true
https://blog.naver.com/inno06039/223138295860?fromRss=true
https://blog.naver.com/inno06039/223138262988?fromRss=true
https://blog.naver.com/inno06039/223138232432?fromRss=true
https://blog.naver.com/inno06039/223087957533?fromRss=true
https://blog.naver.com/inno06039/223066566994?fromRss=true
https://blog.naver.com/inno06039/223063132479?fromRss=true
https://blog.naver.com/inno06039/223061364670?fromRss=true
https://blog.naver.com/inno06039/223030808870?fromRss=true
https://blog.naver.com/inno06039/223012499548?fromRss=true
https://blog.naver.com/inno06039/223009122523?fromRss=true
https://blog.naver.com/inno06039/223008999284?fromRss=tr

---

## 최종 크롤링

### CURL로 구조 확인

In [75]:
import requests
import json

cookies = {
    'BA_DEVICE': '4a497212-b30c-4459-b1ee-a073cda734a7',
    'NNB': 'H2NEZNQYHGVWK',
}

headers = {
    'authority': 'm.blog.naver.com',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'referer': 'https://m.blog.naver.com/PostList.nhn?blogId=inno06039&directAccess=true&tab=1&userTopListOpen=True&userTopListCount=30&userTopListCurrentPage=1',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
}

params = {
    'categoryNo': '0',
    'itemCount': '270',
    'page': '1',
    'userId': '',
}

response = requests.get(
    'https://m.blog.naver.com/api/blogs/inno06039/thumbnail-post-list',
    params=params,
    cookies=cookies,
    headers=headers,
)

# JSON 형식으로 응답을 파싱
data = response.json()

# JSON 구조를 보기 쉽게 출력
print(json.dumps(data, indent=4, ensure_ascii=False))


{
    "isSuccess": true,
    "result": {
        "page": 1,
        "categoryNo": 0,
        "categoryName": "전체글",
        "totalCount": 0,
        "items": [
            {
                "blogId": "inno06039",
                "logNo": 223315892193,
                "titleWithInspectMessage": "2024년 빅데이터 동아리 투빅스 [TOBIGS] 나름의 지극히 주관적인!! 합격 후기? (느끼는 점 등등..)",
                "categoryName": "교내/교외 활동",
                "categoryNo": 15,
                "addDate": 1704686492300,
                "scraped": false,
                "openType": 2,
                "categoryOpenYn": true,
                "categoryBlockYn": false,
                "postBlocked": false,
                "domainIdOrBlogId": "inno06039",
                "thumbnailUrl": "https://mblogthumb-phinf.pstatic.net/MjAyNDAxMDhfNzAg/MDAxNzA0Njg2Mzk5MDI3.WPtyql0wWj1_rFhJ8jpyCJdkB4B4F_CNqS090iPy1jwg.jsQLfQ2_xkb_j_EcEGyfq1_x6vg-cdli7iXCBQyUInog.PNG.inno06039/image.png?type=ffn300_300",
                "videoThumbnail": false,
    

### 바뀌는 부분만 데이터 추출

In [2]:
import requests

cookies = {
    'BA_DEVICE': '4a497212-b30c-4459-b1ee-a073cda734a7',
    'NNB': 'H2NEZNQYHGVWK',
}

headers = {
    'authority': 'm.blog.naver.com',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'referer': 'https://m.blog.naver.com/PostList.nhn?blogId=inno06039&directAccess=true&tab=1&userTopListOpen=True&userTopListCount=30&userTopListCurrentPage=1',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
}

params = {
    'categoryNo': '0',
    'itemCount': '270',
    'page': '1',
    'userId': '',
}

response = requests.get(
    'https://m.blog.naver.com/api/blogs/inno06039/thumbnail-post-list',
    params=params,
    cookies=cookies,
    headers=headers,
)

# JSON 형식으로 응답을 파싱
data = response.json()


# 각 게시물의 "logNo"와 "addDate"를 추출
log_date = [(item['logNo'], item['addDate']) for item in data['result']['items']]

# 결과 출력
for logNo, addDate in log_date:
    print(f"logNo: {logNo}, addDate: {addDate}")



logNo: 223315892193, addDate: 1704686492300
logNo: 223314262954, addDate: 1704522486293
logNo: 223313566231, addDate: 1704445723898
logNo: 223296652000, addDate: 1702901933619
logNo: 223156524687, addDate: 1689342971074
logNo: 223143200809, addDate: 1688110878563
logNo: 223142177846, addDate: 1688021944798
logNo: 223138295860, addDate: 1687679564445
logNo: 223138262988, addDate: 1687676360860
logNo: 223138232432, addDate: 1687673339227
logNo: 223087957533, addDate: 1682673474509
logNo: 223066566994, addDate: 1680751589979
logNo: 223063132479, addDate: 1680484509089
logNo: 223061364670, addDate: 1680274395388
logNo: 223030808870, addDate: 1677583196231
logNo: 223012499548, addDate: 1676098256069
logNo: 223009122523, addDate: 1675835803547
logNo: 223008999284, addDate: 1675828498295
logNo: 222990607502, addDate: 1674301181728
logNo: 222972099894, addDate: 1672575025600
logNo: 222970386613, addDate: 1672397856835
logNo: 222953787583, addDate: 1670851280142
logNo: 222945236353, addDate: 16

In [3]:
len(log_date)

267

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime  # datetime 모듈 import

# 데이터를 저장할 리스트 초기화
titles = []
dates = []
contents = []

for logNo, addDate in log_date:  # 각 logNo와 addDate에 대해서
    params = {
        'blogId': 'inno06039',
        'logNo': str(logNo),  # logNo를 문자열로 변환
        'categoryNo': '1',
        'parentCategoryNo': '1',
        'from': 'thumbnailList',
    }
    response = requests.get('https://blog.naver.com/PostView.naver', params=params, cookies=cookies, headers=headers)

    # BeautifulSoup를 사용하여 HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser')

    # 초기 변수 설정
    current_title = None
    current_content = []

    # 모든 <p> 태그를 찾아서 데이터 추
    p_tags = soup.find_all('p')

    for tag in p_tags:
        if 'class' in tag.attrs and 'se-text-paragraph-align-' in tag['class']:
            if current_title is None:
                current_title = tag.text.strip()
            else:
                current_content.append(tag.text.strip())
        elif 'class' in tag.attrs and 'se-text-paragraph-align-center' in tag['class']:
            if current_title is not None:
                current_content.append(tag.text.strip())

    # "addDate"를 읽기 편한 형식으로 변환
    current_date = datetime.fromtimestamp(addDate / 1000).strftime('%Y-%m-%d %H:%M:%S')

    # 제목, 날짜, 내용 추가
    if current_title is not None:
        titles.append(current_title)
        dates.append(current_date)
        contents.append(' '.join(current_content).replace('\u200b', ''))

# 데이터프레임 생성
data = {'제목': titles, '날짜': dates, '내용': contents}
df = pd.DataFrame(data)
df


,제목,날짜,내용
0,2024년 빅데이터 동아리 투빅스 [TOBIGS] 나름의 지극히 주관적인!! 합격 ...,2024-01-08 13:01:32,나름 이번에 21기를 준비하면서 느꼈는데 생각보다 정보가 없어서..? 간단하게 후기...
1,[SESAC] 청년취업사관학교(도봉 캠퍼스) 준비 과정 및 일주일 후기,2024-01-06 15:28:06,여차저차 하면서 시작하게 된 청년취업사관학교 후기를 좀 남겨볼까해요. 아직 일주일 ...
2,나의 뇌는 4개이다? [나를 알고 싶을 때 뇌과학을 공부합니다.],2024-01-05 18:08:43,오늘은 최근에 읽게 된 챗북에서 인상깊었던 내용이 있어서 다루어보려 합니다. 한줄로...
3,"간만에 써보는, 인생 전체는 되는대로",2023-12-18 21:18:53,"학회 회장 인수인계 자료를 만들다가 문득 간만에 기록을 해본다. 9월, 10월 ,..."
4,Etri 랩실? 하계 인턴 2주 해보고 느낀 것들 정리해보기,2023-07-14 22:56:11,음..이런 느낌의 사회생활은 처음이기에 여러므로 느끼는 바가 많다. 신기하면서도? ...
...,...,...,...
262,1인방송장비(유튜버장비)로는 가성비 최고! - 마타듀얼핀마이크 솔직 구입 후기,2020-12-30 19:01:16,최근들어 유튜브와 같은 1인방송을 시작하시는 분이 많이 늘어났는데요! 아마 유튜브...
263,반드시 알아야 할 카드뉴스 제작 방법!,2020-11-04 18:49:41,본 내용에 들어가기에 앞서 카드뉴스 제작에 관한 정보를 알기위해 들어오셨다면? 카...
264,이미지 사이트 추천 BEST 5,2020-10-11 14:46:27,여러분이 만약 누군가가 만들어 놓은 영상 혹은 게시물을 처음 접했을 때 아무리 그 ...
265,통계 사이트 쉽게 정리하기!,2020-10-04 13:44:51,지식이 많다 못해서 이제는 넘쳐흐르는 요즘 같은 시대에 정확한 정보를 보고 고를 수...


In [5]:
df.to_csv("./blog_data.csv",index=False, encoding="utf-8-sig")

In [6]:
import pandas as pd
data = pd.read_csv('./blog_data.csv')

In [10]:
empty_contents = data[data['내용'].isnull()]
empty_contents

,제목,날짜,내용
71,Daily 북 / < 진짜 공부> 3 Point / (5월 25일),2022-05-25 08:11:05,NaN
77,Daily 북 / < 살아가는데 가장 많이 써먹는 심리학 > 3 Point / (5...,2022-05-23 08:06:13,NaN
188,빅데이터분석기사 필기 정리(PART 02 빅데이터 탐색 - 데이터 탐색),2022-03-07 18:17:51,NaN
224,빅데이터분석기사 필기 정리 (Part 1 빅데이터 분석 기획 - 데이터 분석 계획...,2022-02-03 10:59:36,NaN
249,ERP물류1급 이론(필기)정리 <chp 01. 영업관리>,2022-01-12 12:46:41,NaN


In [12]:
df = data.dropna(subset=['내용']).reset_index(drop=True)

In [15]:
pwd

'c:\\Users\\inho0\\OneDrive\\문서\\GitHub\\giminopt'

In [16]:
df.to_csv("./blog_data.csv",index=False, encoding="utf-8-sig")

---

## 단일 데이터로 파싱 확인

In [44]:
url = 'https://blog.naver.com/inno06039'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [54]:
import requests

cookies = {
    'NNB': 'LV2OKWGZYTKGI',
    'BA_DEVICE': 'f4bdeb60-a7f8-463a-97f1-ea373712cdaa',
    'ASID': '3d63058a0000018ac102a4e600000044',
    '_ga': 'GA1.2.1346474311.1698760006',
    'wcs_bt': '1cde97b94dd8ef0:1699617778',
    'ba.uuid': '84876cdc-3ad7-42bf-8d1a-9e9066770a36',
    'nx_ssl': '2',
    'nid_inf': '861257409',
    'NID_AUT': 'nyzvw87K9tVRdAn1EXIm6HqRkUTlzTk60VWD8uTr/ioJomQ9FMdPx+l8MdK5qyfM',
    'NID_JKL': '/+eQwUJQOJJDHbdvUBbxmdXgiRhgkR2pT+sCa5lEy0s=',
    'stat_yn': '1',
    'page_uid': 'ijpwxwqVOsossAqKSRwssssssxK-096064',
    'NID_SES': 'AAAB2C5xolGJursxLnVl+dauEbNxFUvtcVbCK7Joxb89roDkidh9tcYHOqTexN70LwsN9LG25FawOCFbFZYrsNeCEdGvloEh7UXanvEtQP6MWeAe2C1+0+ksJCugjjLIkIhgkadaUV2hu6YdkvATsnv8MVKYBDOrWMCF/xQCAbALf38w3ll6sxxpN/MZq8cgLAfYfsA/du4dz59Na28eEBUqSccnv5sAYnBYbWhzJutYAMnmwztIPXhz14TOF2P7VduohZe1LUefb5i2aGm8/9+wHjZoJZuqNHAnQ5nfqb101Ybft1Lnjwq4WBjzYSnaflXj5/suZKN8HRU5b5pSzN7UrWXUdplj3zVyapxexMqbaynTwWahrBTGPfzPHfhJqKVQBAOLxgXGmBUVvMEub+U30kaEPa37QiviFX4h83YMLtsf1GYi8M0FNYIP2RjaoZRsVFvcFeUeHGy3Al6Qv69inVvXJcTS7/2sq7luMIVfouvBVNaMzwBPGoRguROJN0V5yzxf2RqLoEeD4CMsm3mCKYtUEqI2AlWpFJYlitfGvtz1CuVXUCS0LY9Jt8aNhIC4bUkJRmstVgh8ymqqP5BbEo04WyQ7PcA8wLlCzfKm3ukY2QKKDXYxZ2Uyto70TPIdLw==',
    'JSESSIONID': '276581343F9B0C6B22F7F302337C9AE3.jvm1',
}

headers = {
    'authority': 'blog.naver.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    # 'cookie': 'NNB=LV2OKWGZYTKGI; BA_DEVICE=f4bdeb60-a7f8-463a-97f1-ea373712cdaa; ASID=3d63058a0000018ac102a4e600000044; _ga=GA1.2.1346474311.1698760006; wcs_bt=1cde97b94dd8ef0:1699617778; ba.uuid=84876cdc-3ad7-42bf-8d1a-9e9066770a36; nx_ssl=2; nid_inf=861257409; NID_AUT=nyzvw87K9tVRdAn1EXIm6HqRkUTlzTk60VWD8uTr/ioJomQ9FMdPx+l8MdK5qyfM; NID_JKL=/+eQwUJQOJJDHbdvUBbxmdXgiRhgkR2pT+sCa5lEy0s=; stat_yn=1; page_uid=ijpwxwqVOsossAqKSRwssssssxK-096064; NID_SES=AAAB2C5xolGJursxLnVl+dauEbNxFUvtcVbCK7Joxb89roDkidh9tcYHOqTexN70LwsN9LG25FawOCFbFZYrsNeCEdGvloEh7UXanvEtQP6MWeAe2C1+0+ksJCugjjLIkIhgkadaUV2hu6YdkvATsnv8MVKYBDOrWMCF/xQCAbALf38w3ll6sxxpN/MZq8cgLAfYfsA/du4dz59Na28eEBUqSccnv5sAYnBYbWhzJutYAMnmwztIPXhz14TOF2P7VduohZe1LUefb5i2aGm8/9+wHjZoJZuqNHAnQ5nfqb101Ybft1Lnjwq4WBjzYSnaflXj5/suZKN8HRU5b5pSzN7UrWXUdplj3zVyapxexMqbaynTwWahrBTGPfzPHfhJqKVQBAOLxgXGmBUVvMEub+U30kaEPa37QiviFX4h83YMLtsf1GYi8M0FNYIP2RjaoZRsVFvcFeUeHGy3Al6Qv69inVvXJcTS7/2sq7luMIVfouvBVNaMzwBPGoRguROJN0V5yzxf2RqLoEeD4CMsm3mCKYtUEqI2AlWpFJYlitfGvtz1CuVXUCS0LY9Jt8aNhIC4bUkJRmstVgh8ymqqP5BbEo04WyQ7PcA8wLlCzfKm3ukY2QKKDXYxZ2Uyto70TPIdLw==; JSESSIONID=276581343F9B0C6B22F7F302337C9AE3.jvm1',
    'referer': 'https://blog.naver.com/PostList.naver?blogId=inno06039&from=postList&categoryNo=1&parentCategoryNo=1',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'iframe',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
}

params = {
    'blogId': 'inno06039',
    'logNo': '223314262954',
    'categoryNo': '1',
    'parentCategoryNo': '1',
    'from': 'thumbnailList',
}
response = requests.get('https://blog.naver.com/PostView.naver', params=params, cookies=cookies, headers=headers)

In [289]:
from bs4 import BeautifulSoup
import pandas as pd

# BeautifulSoup를 사용하여 HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# 데이터를 저장할 리스트 초기화
titles = []
dates = []
contents = []

# 초기 변수 설정
current_title = None
current_date = None

# 모든 <p> 태그와 <span> 태그를 찾아서 데이터 추출
p_tags = soup.find_all('p')
span_tags = soup.find_all('span', class_='date pcol2')

for tag in p_tags:
    # <p class="se-text-paragraph se-text-paragraph-align-"> 태그는 제목으로 저장
    if 'class' in tag.attrs and 'se-text-paragraph-align-' in tag['class']:
        if current_title is None:
            current_title = tag.text.strip()
        else:
            # 이미 다른 제목을 가지고 있으면 이전 제목의 내용 저장
            contents.append(tag.text.strip())
    # <p class="se-text-paragraph se-text-paragraph-align-center"> 태그는 내용으로 저장
    elif 'class' in tag.attrs and 'se-text-paragraph-align-center' in tag['class']:
        if current_title is not None:
            contents.append(tag.text.strip())

for span_tag in span_tags:
    # <span class="date pcol2"> 태그에서 날짜 데이터 추출
    date_text = span_tag.text.strip()
    if date_text:
        current_date = date_text

# 마지막 제목과 내용 추가
if current_title is not None:
    titles.append(current_title)
    dates.append(current_date)
    contents_str = ' '.join(contents)
    contents_str = contents_str.replace('\u200b', '') 
    contents.append(contents_str)
    contents = [' '.join(contents)]

data = {'제목': titles, '날짜': dates, '내용': contents}
df = pd.DataFrame(data)
df


,제목,날짜,내용
0,[SESAC] 청년취업사관학교(도봉 캠퍼스) 준비 과정 및 일주일 후기,2022. 6. 15.,여차저차 하면서 시작하게 된 청년취업사관학교 후기를 좀 남겨볼까해요. 아직 일주일 ...


In [290]:
contents

['여차저차 하면서 시작하게 된 청년취업사관학교 후기를 좀 남겨볼까해요. 아직 일주일 차이긴 하지만..! 느낀 바를 담아보도록 하겠습니다~~ \u200b 1.청년취업사관학교란? 위의 홈페이지에도 나와있듯이 서울시에서 주관하는 일종의 부트캠프?라고 생각하면 편하실 것 같아요. 하지만 해당 과정의 특별한 점이라면 \u200b 국비지원 즉 KDT수업을 들었더라도 이에 상관없이 서울에 거주하는 만 15세 이상이면 무료로! 교육을 수강할 수 있다는 점!!! \u200b 그리고 실제 기업들에서 필요로 하는 것들을 조사한 후에 이를 바탕으로 커리큘럼 및 프로젝트를 진행한다는 점! 이 특별한 것 같아요. \u200b 이 사업은 서울시에서 진행된지 한 3~4년 밖에 안된 사업으로 앞으로 더욱 확장이 될 예정이라고 합니다..! 본 이미지는 사이트에 있는 2023년 자료이니 참고해주세요. 2. 준비 과정 참고로 과정도 엄청 다양하니 자세한 조건을 사이트를 직접 참고하심이 좋을 것 같습니다. 저는 LLM교육과정을 신청했는데 아래와 같습니다. 우선 해당 과정을 신청하게 된 계기는 인공지능 분야 자체가 공부해야 할 것이 음청 많다는 점..? 저 역시 적지 않게 노력은 했으나 먼가 취준 해보면서도 부족한 부분을 많이 느껴서(대학원 가야..허..나) 좀 더 단단하게 지식을 채우고 취업 준비도 도움을 받고 싶어서 지원을 하게 되었답니다. \u200b 선발되는 과정은 1) 구글 폼으로 자기소개서 작성 2) 간단한 코딩 테스트 3) 10~15분 간 진행되는 대면 면접 (다대일) 이라고 보시면 될 것 같아요. \u200b 이 3단계에서 가장 중요한 것은 면접이 아닐까 싶어요. 왜냐하면 저의 경우 자기소개폼에 자기소개와 지원동기 300자 정도? 밖에 쓰는 것이 없었고 면접 일정 역시 코딩 테스트 바로 다음일자 중으로 있어서 지원자가 이 과정을 성실히 할 사람인가를 중심으로 보신거 같더라구요. \u200b 실제로 면접관 님께서도 코딩테스트는 지원자의 지식 수준 파악을 위한 것으로 합격 여부에 크게 

In [252]:
strings_to_remove = [
    '\u200b',
    '안녕하세요.이 포스트는 네이버 블로그에서 작성된 게시글입니다.자세한 내용을 보려면 링크를 클릭해주세요.감사합니다.',
    '2009년 6월 30일 네이버 여행 서비스가 종료되었습니다. 네이버 여행 서비스를 이용해 주신 여러분께 감사드리며, 더 좋은 서비스로 보답할 수 있도록 노력하겠습니다.',
    '{FILENAME}',
    '백신 프로그램으로 치료하신 후 다시 첨부하시거나, 치료가 어려우시면파일을 삭제하시기 바랍니다.',
    '고객님의 PC가 악성코드에 감염될 경우 시스템성능 저하,개인정보 유출등의 피해를 입을 수 있으니 주의하시기 바랍니다.',
    '{ALERTMESSAGE}',
    '이용제한 파일 : {FILENAME}',
    '네이버는 블로그를 통해 저작물이 무단으로 공유되는 것을 막기 위해,\n\t\t\t\t\t\t저작권을 침해하는 컨텐츠가 포함되어 있는 게시물의 경우 글보내기 기능을 제한하고 있습니다.',
    '상세한 안내를 받고 싶으신 경우 네이버 고객센터로 문의주시면 도움드리도록 하겠습니다.\n\t\t\t\t\t\t건강한 인터넷 환경을 만들어 나갈 수 있도록 고객님의 많은 관심과 협조를 부탁드립니다.',
    '네이버는 블로그를 통해 저작물이 무단으로 공유되는 것을 막기 위해,\n\t\t\t\t\t\t저작권을 침해하는 컨텐츠가 포함되어 있는 게시물의 경우 주제 분류 기능을 제한하고 있습니다.',
    '상세한 안내를 받고 싶으신 경우 네이버 고객센터로 문의주시면 도움드리도록 하겠습니다.\n\t\t\t\t\t\t건강한 인터넷 환경을 만들어 나갈 수 있도록 고객님의 많은 관심과 협조를 부탁드립니다.',
    '작성하신 게시글에 사용이 제한된 문구가 포함되어 일시적으로  등록이 제한됩니다.',
    '이용자 분들이 홍보성 도배, 스팸 게시물로 불편을 겪지 않도록 다음과 같은 경우 해당 게시물 등록이 일시적으로 제한됩니다.',
    '스팸 게시물이 확대 생성되는 것을 방지하기 위하여 문구 및 사용 제한기간을상세하게 안내해 드리지 못하는 점 양해 부탁 드립니다. 모두가 행복한 인터넷문화를 만들기 위한 네이버의 노력이오니 회원님의 양해와 협조 부탁드립니다.',
    '더 궁금하신 사항은 고객센터로 문의하시면 자세히 알려드리겠습니다.',
    '수정하신 후 다시 등록해 주세요.',
    '회원님의 안전한 서비스 이용을 위해 비밀번호를 확인해 주세요.',
    '다시 한번 비밀번호 확인 하시면 이용중인 화면으로 돌아가며, 작성 중이던내용을 정상적으로 전송 또는 등록하실 수 있습니다.',
    '',
    '이 글의 공감수도 함께 차감됩니다.',
    '작성하신 에 이용자들의 신고가 많은 표현이 포함되어 있습니다.',
    '다른 표현을 사용해주시기 바랍니다.\n\t\t\t건전한 인터넷 문화 조성을 위해 회원님의 적극적인 협조를 부탁드립니다.',
    '더 궁금하신 사항은 고객센터로 문의하시면 자세히 알려드리겠습니다.',
    '내 상품 관리에서 배송비 설정 후 상품 판매를 시작해보세요!']

In [259]:
df['내용']

0    아직 일주일 차이긴 하지만..! 느낀 바를 담아보도록 하겠습니다~~ 위의 홈페이지에...
Name: 내용, dtype: object

In [211]:
p_tags

[<p class="desc">
                     블로그 마켓 판매자의 이력 관리를 위해<br/>
                     블로그 주소 변경이 불가합니다.
                 </p>,
 <p class="desc">
                     블로그에서 진짜 나를 기록하고<br/>
                     다양한 이웃과 소식을 만나보세요. 지금 시작해볼까요?
                 </p>,
 <p class="text_small">
                     1. 이전 주소로 공유된 글은<br/>
                     3개월간 새로운 주소로 연결을 지원하며<br/>
                     이후 언제든 연결이 끊길 수 있습니다.
                 </p>,
 <p class="text_small">
                     2. 블로그 아이디는 한번 변경하면<br/>
 <strong class="red_color">다시 변경이 불가능</strong>합니다.
                 </p>,
 <p class="text_small">
 <strong>변경하시겠습니까?</strong>
 </p>,
 <p class="text_red">
                     블로그 아이디는 한번 정하면
                     <br/>
                     다시 변경이 불가능합니다.
                 </p>,
 <p class="text">이 아이디로 블로그를 만들까요?</p>,
 <p class="title_desc">나중에 언제든지 변경할 수 있어요.</p>,
 <p class="title_desc">선택한 분야의 글과 이웃을 추천받을 수 있어요.</p>,
 <p class="title_desc">이웃을 맺으면 이웃새글에서 글을 받아볼 수 있어요.</p>,
 <p clas

In [210]:
soup.find_all('span')

[<span class="icon_category_group">
 <i class="line"></i>
 <i class="line"></i>
 <i class="line"></i>
 </span>,
 <span class="u_ico _icon"></span>,
 <span class="u_likeit_list_btn _button" data-ishiddenzerocount="true" data-log="pst.flike|pst.funlike" data-type="like" role="button">
 <em class="u_txt _label">공감</em>
 <em class="u_cnt _count"></em>
 </span>,
 <span class="comment_wrap">
 <em id="floating_bottom_comment_label">
 										
 										
 											댓글
 										
 									</em>
 <em id="floating_bottom_commentCount">
 										
 											13
 										
 									</em>
 </span>,
 <span class="blind">공유하기</span>,
 <span class="blogname" data-content="blogName"></span>,
 <span class="category" data-content="themeName"></span>,
 <span class="ico_arrow"></span>,
 <span class="blind">블로그 메뉴</span>,
 <span class="pcol2"></span>,
 <span class="cline"></span>,
 <span class="pcol3"></span>,
 <span class="comment"></span>,
 <span class="btn"></span>,
 <span class="pcol2b"></span>

---